# Template 4-Player Analysis
このノートブックは `logs/` 以下の JSONL を読み込み、
簡易的な勝率を確認するためのテンプレートです。
必要に応じてセルを追加して詳細分析へ拡張してください。


In [3]:
from pathlib import Path
import json
import pandas as pd

LOG_PATH = Path('../logs/logfile.jsonl')

if not LOG_PATH.exists():
    raise FileNotFoundError(f'ログが見つかりません: {LOG_PATH}')

records = []
with LOG_PATH.open('r', encoding='utf-8') as fh:
    for line in fh:
        line = line.strip()
        if not line:
            continue
        records.append(json.loads(line))

runs = pd.DataFrame([r for r in records if r.get('phase') == 'vote_summary'])
if runs.empty:
    raise ValueError('vote_summary が含まれるログがありません。')
runs['run'] = runs['run'].astype(int)
runs = runs.sort_values('run').reset_index(drop=True)
runs


,timestamp,run,round,phase,votes,tally
0,2025-10-18T13:36:46.055462+00:00,1,3,vote_summary,"[{'agent': 'A', 'vote': 'B'}, {'agent': 'B', '...","{'B': 1, 'A': 1, 'D': 1, 'C': 1}"
1,2025-10-18T13:39:33.945118+00:00,2,3,vote_summary,"[{'agent': 'A', 'vote': 'C'}, {'agent': 'B', '...","{'C': 1, 'A': 3}"


In [4]:
def winner_from_summary(row):
    tally = row['tally']
    if not tally:
        return 'no_vote'
    max_votes = max(tally.values())
    top = [player for player, count in tally.items() if count == max_votes]
    if len(top) == 1:
        return top[0]
    return 'tie'

runs['eliminated'] = runs.apply(winner_from_summary, axis=1)
elimination_counts = runs['eliminated'].value_counts()
total_runs = len(runs)
display(pd.DataFrame({'count': elimination_counts, 'rate': elimination_counts / total_runs}))


,count,rate
eliminated,,
tie,1,0.5
A,1,0.5
